# 振子1实验：

## 使用SINDY测试

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import pysindy as ps
from pysindy.feature_library import GeneralizedLibrary, PolynomialLibrary, FourierLibrary
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso

# 自定义特征函数
def x_cos_x(x):
    return x * np.cos(x)

def x_sin_x(x):
    return x * np.sin(x)

def exp_x(x):
    return np.exp(x)

# 修复 function_names 参数，确保其为字符串列表
custom_library = ps.CustomLibrary(
    library_functions=[x_cos_x, x_sin_x, exp_x],
    function_names=[lambda x: "x*cos(x)", lambda x: "x*sin(x)", lambda x: "exp(x)"]  # 使用 lambda 确保正确的字符串表示
)

# 1. 读取数据
df = pd.read_csv('./test_ood.csv')
x = df['x'].values
v = df['v'].values
a = df['a'].values

X = np.vstack((x, v)).T
y = a.T

# 2. 构建并拟合 SINDy 模型
#    - 用多项式库（degree=5）来捕捉 sin(theta) 展开
#    - 用 STLSQ 稀疏回归
poly_lib = PolynomialLibrary(degree=4)            # [1, x, x^2, x^3]
fourier_lib = FourierLibrary(n_frequencies=2)      #  [1, sin(x), cos(x), sin(2x), cos(2x)]
library = GeneralizedLibrary(libraries=[poly_lib, fourier_lib, custom_library])
#optimizer = Lasso(alpha=0.001)  # 设置正则化强度
#optimizer = ps.STLSQ(threshold=0.1, alpha=0.01)  # 稀疏回归
optimizer = ps.SR3(threshold=0.1)  # 稀疏回归
model = ps.SINDy(feature_library=library, optimizer=optimizer)
model.fit(X, x_dot=a)
model.print()  # 打印识别出的方程

# 使用模型预测
predicted_values = model.predict(X)

# 计算均方误差 (MSE)
mse = mean_squared_error(a, predicted_values)
print(f"均方误差 (MSE): {mse}")



In [ ]:
# 1. 读取数据
df = pd.read_csv('./train.csv')
x = df['x'].values
v = df['v'].values
a = df['a'].values

X = np.vstack(( x, v)).T
y = a.T

predicted_values = model.predict(X)

# 计算均方误差 (MSE)
mse = mean_squared_error(a, predicted_values)
print(f"均方误差 (MSE): {mse}")

# 使用PYSR测试

In [2]:
import numpy as np
import pandas as pd
from pysr import PySRRegressor
import matplotlib.pyplot as plt

# 1. 读取数据
df = pd.read_csv('./test_ood.csv')
x = df['x'].values
v = df['v'].values
a = df['a'].values

X = np.vstack((x, v)).T
y = a.T

# 定义 PySR 模型
model = PySRRegressor(
    niterations=100,  # 迭代次数
    populations=30,  # 种群数量
    binary_operators=["+", "-", "*", "/","^"],  # 二元操作符
    unary_operators=["sin", "cos","exp"],  # 一元操作符
    nested_constraints={
                "sin": {"cos": 0,"sin":0},  # 禁止 sin(cos(x))
                "exp": {"exp": 0,"cos": 0,"sin":0},  # 禁止 exp(exp(x))
                "cos": {"sin": 0,"cos": 0},
            },
    constraints={
        "^": (-1, 1)  # 再次尝试限制指数为常数
    },
    elementwise_loss="L2DistLoss()",  # 损失函数
    parallelism='multithreading',  # 启用多线程
    verbosity=1,  # 显示详细日志 (verbosity 而不是 verbose)
)

# 拟合模型
print("Fitting model...")
model.fit(X, y)

# 输出最佳表达式
for i, expr in enumerate(model.equations_):
    print(f"Best Expression for output {i+1}: {equation['equation']}")




Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython
Fitting model...
Compiling Julia backend...


c:\anacoda\envs\pytorch\Lib\site-packages\pysr\sr.py:2737: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\anacoda\envs\pytorch\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 1.420e+04
Progress: 79 / 3000 total iterations (2.633%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           7.598e-03  1.594e+01  y = -0.0073098
3           3.476e-03  3.911e-01  y = x₀ * -0.088855
4           3.286e-03  5.612e-02  y = sin(x₀) * -0.10347
5           3.178e-03  3.354e-02  y = (-0.092868 - x₁) * x₀
6           2.810e-03  1.228e-01  y = sin(x₀ / 0.45604) * -0.085045
7           2.040e-03  3.202e-01  y = ((-0.43471 - x₁) * 0.18229) * x₀
8           1.948e-03  4.620e-02  y = (sin(x₁ / -0.32444) * x₁) * x₀
9           1.016e-03  6.510e-01  y = ((-0.87885 - x₁) + 0.65054) * (x₀ * 0.42003)
12          5.647e-04  1.958e-01  y = (0.14245 / (-0.48974 / (x₁ - -0.34893))) * sin(x₀ + x₀...
                                      )
14          2.776e-

[ Info: Final population:
[ Info: Results saved to:


───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           7.598e-03  1.594e+01  y = -0.0073098
3           3.476e-03  3.911e-01  y = x₀ * -0.088855
4           3.286e-03  5.621e-02  y = sin(x₀) * -0.10429
5           3.080e-03  6.473e-02  y = (x₀ + x₁) * -0.091219
6           2.227e-03  3.243e-01  y = (-0.10932 - x₁) * sin(x₀)
7           9.129e-04  8.916e-01  y = ((-0.17639 - x₁) * x₀) * 0.51546
8           7.125e-04  2.479e-01  y = (x₁ + 0.18932) * (sin(x₀) * -0.56589)
9           4.333e-04  4.973e-01  y = (((-0.37406 - x₁) * x₁) - 0.072102) * x₀
10          2.287e-04  6.392e-01  y = x₀ * ((x₁ + (cos(x₀) * 0.29065)) * -0.5267)
11          1.491e-04  4.277e-01  y = (sin(x₀) * -0.57926) * (x₁ + (cos(x₀) * 0.29814))
12          6.198e-05  8.778e-01  y = (x₀ * (x₁ + ((cos(x₀) + x₁) + -0.2808))) * -0.26722
13          6.016e-05  2.974e-02  y = (x₀ * ((cos(x₀) + sin(x₁ + -0.29103)) + x₁)) * -0.

In [8]:
# 1. 读取数据
best_equation = model.get_best()
print(f"Best equation: {best_equation}")

from sklearn.metrics import mean_squared_error
df = pd.read_csv('./train.csv')
x = df['x'].values
v = df['v'].values
a = df['a'].values

X = np.vstack(( x, v)).T
y = a.T

predicted_values = model.predict(X)

# 计算均方误差 (MSE)
mse = mean_squared_error(a, predicted_values)
print(f"均方误差 (MSE): {mse}")

c:\anacoda\envs\pytorch\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Best equation: complexity                                                      28
loss                                                      0.000002
equation         (((((x1 + cos(x0 + -0.020016588)) + x1) * cos(...
score                                                     0.088127
sympy_format     ((x1 + x1 + cos(x0 - 0.020016588))*cos(x0*x1 -...
lambda_format    PySRFunction(X=>((x1 + x1 + cos(x0 - 0.0200165...
Name: 21, dtype: object
均方误差 (MSE): 3.383805438751717e-05
